In [45]:
# generate the random words

import random
import os

words = ['Environment', 'Archives', 'Pronounciation', 
         'Hour', 'Wednesday', 'Violence', 'Tomb', 'Suite', 
         'Iron', 'Reciept', 'Chores']
random_words = random.sample(words, 5)
random_string = ' '.join(random_words)
print(random_string)

# Directory path to traverse
directory_path = 'models/'
models = []

for word in random_words:
    for file in os.listdir(directory_path):
        if word in file:
            models.append(file)

Reciept Chores Suite Iron Violence


In [46]:
models

['Reciept_RandomForest_model.joblib',
 'Chores_RandomForest_model.joblib',
 'Suite_RandomForest_model.joblib',
 'Iron_RandomForest_model.joblib',
 'Violence_RandomForest_model.joblib']

In [47]:
# collect audio samples

import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv

# name = input("Enter your name: ")
name = 'Unknown'
freq = 44100
duration = 2

def delete_old_files(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)

delete_old_files("audio_files/test")

words = random_words

for word in words:
    for i in range(1):
        file_name = "audio_files/test/" + name + str(i+1) + word + '.wav'
        print("Recording file " + file_name)
        print(f'{word} - {i+1}')
        recording = sd.rec(int(duration * freq), samplerate=freq, channels=2)
        sd.wait()
        write(file_name, freq, recording)
        print("Recorded file " + file_name)

Recording file audio_files/test/Unknown1Reciept.wav
Reciept - 1
Recorded file audio_files/test/Unknown1Reciept.wav
Recording file audio_files/test/Unknown1Chores.wav
Chores - 1
Recorded file audio_files/test/Unknown1Chores.wav
Recording file audio_files/test/Unknown1Suite.wav
Suite - 1
Recorded file audio_files/test/Unknown1Suite.wav
Recording file audio_files/test/Unknown1Iron.wav
Iron - 1
Recorded file audio_files/test/Unknown1Iron.wav
Recording file audio_files/test/Unknown1Violence.wav
Violence - 1
Recorded file audio_files/test/Unknown1Violence.wav


In [48]:
# generate df for test set
# generate the MFCC from audio files - 
import os
import pandas as pd
import librosa
import re

# Function to extract MFCC features from an audio file
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    return mfccs.tolist()  # Convert to list for easier storage in DataFrame

# Directory containing audio files
directory_path = "audio_files/test"

# Target folder to save DataFrames as CSV files
target_folder = "df/test"

# Create the target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# List of words
words = random_words

# Initialize a dictionary to store DataFrames for each word
word_dataframes = {word: pd.DataFrame() for word in words}

# Iterate over the filenames
for filename in os.listdir(directory_path):
    # Use regular expression to extract name and word
    match = re.match(r'([A-Za-z]+)(\d+)([A-Za-z]+)', filename)
    if match:
        name = match.group(1)
        word = match.group(3)
        num = int(match.group(2))
        
        # Extract MFCC features
        mfccs = extract_mfcc(os.path.join(directory_path, filename))

        # Create a DataFrame for each person
        person_df = pd.DataFrame({"Name": [name], "MFCC": [mfccs]})
        
        # Concatenate with existing DataFrame for the word
        word_dataframes[word] = pd.concat([word_dataframes[word], person_df], ignore_index=True)

# Save each DataFrame to a separate CSV file in the target folder
for word, df in word_dataframes.items():
    output_file_path = os.path.join(target_folder, f"{word}_dataframe.csv")
    df.to_csv(output_file_path, index=False)
    print(f"DataFrame for word '{word}' saved to '{output_file_path}'")

DataFrame for word 'Reciept' saved to 'df/test\Reciept_dataframe.csv'
DataFrame for word 'Chores' saved to 'df/test\Chores_dataframe.csv'
DataFrame for word 'Suite' saved to 'df/test\Suite_dataframe.csv'
DataFrame for word 'Iron' saved to 'df/test\Iron_dataframe.csv'
DataFrame for word 'Violence' saved to 'df/test\Violence_dataframe.csv'


In [49]:
# predict from the df
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import ast

# Directory containing audio files
# directory_path = "audio_files"

# List of words
words = random_words

# Initialize a dictionary to store trained models
trained_models = {}

# Define a list of classifiers
classifiers = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', C=1),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "LogisticRegression": LogisticRegression(max_iter=1000)
}

model = models

for i in range(len(words)):
    # Load the dataframe for the current word
    df = pd.read_csv("df/test/" + f"{words[i]}_dataframe.csv")
    # Convert the string representation of MFCC values back to list of lists
    df["MFCC"] = df["MFCC"].apply(ast.literal_eval)
    # Flatten the MFCC values
    df["MFCC"] = df["MFCC"].apply(lambda x: [item for sublist in x for item in sublist])
    loaded_model = joblib.load("models/" + model[i])
    
    X = np.array(df["MFCC"].tolist())
    user = loaded_model.predict(X)
    print(user)

['Arun']
['Abhishek']
['Sunamdha']
['Arun']
['Sunamdha']
